In [1]:
%pwd

'/Users/waittim/Google 云端硬盘/Mask-detection/mask-detector/modeling'

In [3]:
%pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/e9/1b/6a420d7e6ba431cf3d51b2a5bfa06a958c4141e3189385963dc7f6fbffb6/tensorboard-2.3.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b6/85/5c5ac0a8c5efdfab916e9c6bc18963f6a6996a8a1e19ec4ad8c9ac9c623c/tensorboard_plugin_wit-1.7.0-py3-none-any.whl
ERROR: tensorflow 2.1.0 has requirement tensorboard<2.2.0,>=2.1.0, but you'll have tensorboard 2.3.0 which is incompatible.
  Found existing installation: PyYAML 5.1.1
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import xml.etree.ElementTree as ET
import glob
from PIL import Image
import random

In [5]:
def xml_to_txt(indir, outdir, imagedir):
 
    #os.chdir(indir)
    annotations = os.listdir(indir+'/.')
    #annotations = glob.glob(str(annotations)+'*.xml')
    #print(annotations)
    for i, file in enumerate(annotations):
         
        file_save = file.split('.')[0]+'.txt'
        #print(file_save)
        # file_txt=os.path.join(outdir,file_save)
        file_txt = outdir+"/"+file_save
        f_w = open(file_txt, 'w+')
 
        # actual parsing
        in_file = open(indir+'/'+file)
        tree=ET.parse(in_file)
        root = tree.getroot()
        filename = root.find('filename').text  #Get filename from the root of xml
        
        # Get the image size
        image = Image.open(imagedir+'/'+file.split('.')[0]+'.jpg')
        image_width = image.width
        image_height = image.height
        
        for obj in root.iter('object'):
                current = list()
                name = obj.find('name').text   #get the class name then transfer to number
                if name == 'face':
                    class_number = '0'
                elif name == 'face_mask':
                    class_number = '1'
                else: 
                    class_number = '2' #wrong class
                    print(file_save) # if return any wrong class, get correct it
 
                xmlbox = obj.find('bndbox')
                xmin = int(xmlbox.find('xmin').text)
                xmax = int(xmlbox.find('xmax').text)
                ymin = int(xmlbox.find('ymin').text)
                ymax = int(xmlbox.find('ymax').text)
                
                x_center = str((xmin + xmax) / 2 / image_width)
                y_center = str((ymin + ymax) / 2 / image_height)
                width = str((xmax - xmin) / image_width)
                height = str((ymax - ymin) / image_height)
                
                #print xn
                f_w.write(class_number +' '+x_center+' '+y_center+' '+width+' '+height+'\n')
 
 

In [3]:
indir = './data/xml_labels'   #xml folder
outdir = './data/labels'  #txt folder
imagedir = './data/images'  # image folder

In [7]:
xml_to_txt(indir,outdir,imagedir)

In [4]:
test_percentage = 0.2

all_image = os.listdir(imagedir+'/.')

test_index = list(random.sample(range(0,len(all_image)), round(len(all_image)*test_percentage)))
train_index = [i for i in range(0,len(all_image)) if i not in test_index]

test_data = [all_image[i] for i in test_index]
train_data = [all_image[i] for i in train_index]

In [5]:
print('test:',len(test_data), 'train:', len(train_data))

test: 1811 train: 7245


In [6]:
f_w = open('./data/train.txt', 'w+')
for i in train_data:
    f_w.write('./images/'+i+'\n')
f_w.close()

f_w = open('./data/valid.txt', 'w+')
for i in test_data:
    f_w.write('./images/'+i+'\n')
f_w.close()

In [13]:
f_w = open('./data/face_mask.names.txt', 'w+')
class_list = ['NoMask', 'Mask']
for i in class_list:
    f_w.write(i+'\n')

f_w.close()
    
os.rename('./data/face_mask.names.txt', './data/face_mask.names')

In [11]:
f_w = open('./data/face_mask.data.txt', 'w+')
content = ['classes=2', 
           'train=data/train.txt', 
           'valid=data/valid.txt', 
           'names=data/face_mask.names']
for i in content:
    f_w.write(i+'\n')

f_w.close()
    
os.rename('./data/face_mask.data.txt', './data/face_mask.data')